#imports



In [5]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ipynb
import sys
sys.path.insert(0,'/content/drive/MyDrive/CNN-deterministic-decoupling')
path = '/content/drive/MyDrive/CNN-deterministic-decoupling'
import torch
import math
from ipynb.fs.defs.imageFunctions import prepare_image,show_image,save_image
from ipynb.fs.defs.Runge_Kutta import RungeKutta4
from ipynb.fs.defs.DISTS_mod import DISTS
from PIL import Image
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Hacia atras

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
target = prepare_image(Image.open(path+"/imagenes/texture6.jpg").convert("RGB"),size=128)
target=target.to(device)
target.requires_grad_(False)

#random=torch.normal(mean=0.45, std=0.22,size=target.size())
origin=prepare_image(Image.open(path+"/imagenes/texture5.jpg").convert("RGB"),size=128)
origin=origin.to(device)
origin.requires_grad_(True)
show_image(origin)

x0=origin.detach().clone()
x0=x0.to(device)


model_d = DISTS().to(device)
model_d.set_target(target)

epoch=0
step_size = 0.025

ErrorList=[math.sqrt(((x0-origin)**2).mean())/math.sqrt((x0**2).mean())]
EpochList=[epoch]
inicial_Loss=model_d.forward(origin).item()
LossList=[inicial_Loss]
epoch+=1

while LossList[-1] > 0.1:
    RungeKutta4(origin,model_d.forward,step_size,LossList)
    error = math.sqrt(((x0-origin)**2).mean())/math.sqrt((x0**2).mean())
    ErrorList.append(error)
    EpochList.append(epoch)
    epoch+=1
    if epoch%1000==0:
      show_image(origin)
      print("iteracion "+str(epoch))
      print("loss "+str(LossList[-1]))
      with open(path+'/Datos/datos_RK4_6_4.txt', 'w') as f:
        f.write(f'Total iterations: {epoch}\n')
        f.write('\n')
        f.write(f'Error values: {ErrorList}\n')
        f.write('\n')
        f.write(f'Loss values: {LossList}\n')


print("END 1")
print("iteracion "+str(epoch))
save_image(origin,path+'/Datos/turning_RK4_6_4.png')
show_image(origin)
last_error=ErrorList[-1]
print("error"+str(last_error))

model_d.set_target(x0)
error_count=0
error_margin=10

while error_count < error_margin:

  if ErrorList[-1] > last_error:
    error_count += 1
  else:
    error_count = 0

  last_error=ErrorList[-1]
  RungeKutta4(origin,model_d.forward, step_size,LossList)
  error = math.sqrt(((x0-origin)**2).mean())/math.sqrt((x0**2).mean())
  ErrorList.append(error)
  EpochList.append(epoch)
  epoch+=1

  if epoch%2000==0:
    print("iteracion "+str(epoch))
    show_image(origin)
    print("iteracion "+str(epoch))
    print("loss "+str(LossList[-1]))
    with open(path+'/Datos/datos_RK4_6_4.txt', 'w') as f:
      f.write(f'Total iterations: {epoch}\n')
      f.write('\n')
      f.write(f'Error values: {ErrorList}\n')
      f.write('\n')
      f.write(f'Loss values: {LossList}\n')
    save_image(origin,path+'/Datos/current__RK4_6_4.png')


print("END2")
print("iteracion "+str(epoch))


In [ ]:
plt.plot(EpochList,ErrorList)
print((ErrorList[-1]/max(ErrorList))*100)
plt.grid(True)
print(ErrorList[-1])

In [ ]:
plt.plot(EpochList,LossList)
plt.grid(True)

In [ ]:
plt.plot(LossList,ErrorList)
plt.grid(True)